GUI for the following tasks: 
1. Create circuit $U\ket{0}$ and get $f(\alpha) = \braket{0|U^\dagger P U|0}$ 
2. Get the circuit output after applying the Fourier Coefficient Extraction algorithm to $f(\alpha)$

# Part 1

## GUI Code

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

gates = []
observable_terms = []
n_qubits = 1
dim_x = 1
dim_alpha = 1

# ---------------- Top-level widgets ----------------
dim_x_widget = widgets.BoundedIntText(value=1, min=0, max=10, description="dim(x):")
dim_alpha_widget = widgets.BoundedIntText(value=1, min=0, max=10, description="dim(alpha):")
n_qubits_widget = widgets.BoundedIntText(value=1, min=1, max=10, description="Qubits:")
apply_dims_button = widgets.Button(description="Apply", button_style="info")
dims_out = widgets.Output()

# ---------------- Circuit builder widgets ----------------
gate_category = widgets.ToggleButtons(options=["Fixed", "Encoding"], description="Gate Type:", disabled=True)

fixed_gate_widget = widgets.Dropdown(description="Fixed Gate:", disabled=True)
encoding_gate_widget = widgets.Dropdown(description="Encoding Gate:", disabled=True)
encoding_gate_widget.layout.display = "none"

target_qubits_widget = widgets.Text(value="0", description="Target(s):", disabled=True)

add_gate_button = widgets.Button(description="Add Gate", button_style="success", disabled=True)
remove_gate_widget = widgets.Dropdown(description="Remove Gate:", disabled=True)
remove_gate_button = widgets.Button(description="Remove", button_style="danger", disabled=True)

gate_display = widgets.Output()

observable_input = widgets.Text(value="Z0", description="Observable:", disabled=True)
add_obs_button = widgets.Button(description="Add Term", button_style="info", disabled=True)

remove_obs_widget = widgets.Dropdown(description="Remove Obs:", disabled=True)
remove_obs_button = widgets.Button(description="Remove", button_style="danger", disabled=True)

obs_display = widgets.Output()

display_mode = widgets.RadioButtons(
    options=["U", "U P U†"], value="U P U†", description="Display:", disabled=True
)

show_circuit_button = widgets.Button(description="Show Circuit", button_style="warning", disabled=True)
clear_button = widgets.Button(description="Clear All", button_style="danger", disabled=True)

output_display = widgets.Output()


# ---------------- Helper functions ----------------
def show_code_block(code_text):
    html = f"""
    <pre style="background:#f5f5f5; padding:10px; border:1px solid #ccc; font-size:14px;">
{code_text}
    </pre>
    <button onclick="navigator.clipboard.writeText(`{code_text}`)">Copy to Clipboard</button>
    """
    display(widgets.HTML(html))


def update_gate_options():
    """Populate fixed_gate_widget and encoding_gate_widget with options.
    Fixed gates use x[...] parameters; encoding use α[...] parameters.
    We add single-qubit rotations and two-qubit Pauli rotations.
    """
    global fixed_gate_widget, encoding_gate_widget, dim_x, dim_alpha, n_qubits

    paulis = ["X", "Y", "Z"]

    # Base fixed ops
    fixed_ops = ["X", "Y", "Z", "H", "CNOT"]
    for i in range(dim_x):
        fixed_ops += [f"Rx(x[{i}])", f"Ry(x[{i}])", f"Rz(x[{i}])"]

    # Add two-qubit Pauli rotations parameterized by x (fixed)
    # user will specify the target qubits in `target_qubits_widget`
    for p1 in paulis:
        for p2 in paulis:
            for k in range(dim_x):
                fixed_ops.append(f"R{p1}{p2}(x[{k}])")

    # Encoding ops (α)
    enc_ops = []
    for j in range(dim_alpha):
        enc_ops += [f"Rx(α[{j}])", f"Ry(α[{j}])", f"Rz(α[{j}])"]
    for p1 in paulis:
        for p2 in paulis:
            for j in range(dim_alpha):
                enc_ops.append(f"R{p1}{p2}(α[{j}])")

    fixed_gate_widget.options = fixed_ops
    encoding_gate_widget.options = enc_ops


def refresh_gate_display():
    with gate_display:
        clear_output()
        print("Current Gates:")
        for idx, g in enumerate(gates):
            print(f"{idx}: {g}")
    remove_gate_widget.options = [f"{i}: {g['type']} {g['targets']}" for i, g in enumerate(gates)]


def refresh_obs_display():
    with obs_display:
        clear_output()
        print("Current Observable Terms:")
        for idx, o in enumerate(observable_terms):
            print(f"{idx}: {o}")
    remove_obs_widget.options = [f"{i}: {o}" for i, o in enumerate(observable_terms)]


# ---------------- Event handlers ----------------
def apply_dims(b):
    global dim_x, dim_alpha, n_qubits
    dim_x = dim_x_widget.value
    dim_alpha = dim_alpha_widget.value
    n_qubits = n_qubits_widget.value
    update_gate_options()

    for w in [gate_category, fixed_gate_widget, encoding_gate_widget,
              target_qubits_widget, add_gate_button, remove_gate_widget, remove_gate_button,
              observable_input, add_obs_button, remove_obs_widget, remove_obs_button,
              display_mode, show_circuit_button, clear_button]:
        w.disabled = False

    with dims_out:
        clear_output()
        print(f"Applied: dim(x)={dim_x}, dim(alpha)={dim_alpha}, qubits={n_qubits}")


def update_gate_dropdown(change):
    if gate_category.value == "Fixed":
        fixed_gate_widget.layout.display = "block"
        encoding_gate_widget.layout.display = "none"
    else:
        fixed_gate_widget.layout.display = "none"
        encoding_gate_widget.layout.display = "block"


def parse_targets(txt):
    txt = txt.strip()
    if txt == "":
        return []
    parts = txt.split()
    targets = []
    for p in parts:
        try:
            targets.append(int(p))
        except:
            raise ValueError(f"Invalid target index: {p}")
    return targets


def add_gate_callback(b):
    gate = fixed_gate_widget.value if gate_category.value == "Fixed" else encoding_gate_widget.value

    try:
        targets = parse_targets(target_qubits_widget.value)
    except Exception as e:
        with gate_display:
            print("❌ Error parsing targets:", e)
        return

    for t in targets:
        if t < 0 or t >= n_qubits:
            with gate_display:
                print(f"❌ Error: target qubit {t} is out of range. Max={n_qubits - 1}")
            return

    gates.append({"type": gate, "targets": targets})
    refresh_gate_display()


def remove_gate_callback(b):
    if len(gates) > 0:
        idx = int(remove_gate_widget.value.split(":")[0])
        gates.pop(idx)
        refresh_gate_display()


def add_obs_callback(b):
    observable_terms.append(observable_input.value)
    refresh_obs_display()


def remove_obs_callback(b):
    if len(observable_terms) > 0:
        idx = int(remove_obs_widget.value.split(":")[0])
        observable_terms.pop(idx)
        refresh_obs_display()


def clear_all(b):
    gates.clear()
    observable_terms.clear()
    refresh_gate_display()
    refresh_obs_display()
    with output_display: clear_output()


def build_P(observable_terms):
    """
    Builds a Pennylane observable P from a list of strings like:
      ["Z0", "X0 Y1", "X0 Z1 + Y2"]
    Returns a valid qml.Observable object.
    """

    P = None

    for term in observable_terms:
        # Remove plus and treat "X0 + Z1" same as separate terms
        pieces = term.replace("+", " ").split()
        local = None

        for p in pieces:
            op = p[0].upper()      # 'X','Y','Z'
            idx = int(p[1:])       # wire index

            t = getattr(qml, f"Pauli{op}")(idx)

            # Tensor product if needed
            local = t if local is None else local @ t

        # Sum if multiple terms
        P = local if P is None else P + local

    # Default: PauliZ(0) if nothing provided
    return P if P is not None else qml.PauliZ(0)


# ----------------- New helper: basis mapping -----------------
def _map_axis_to_Z_forward(axis, wire):
    """Map axis X/Y/Z to Z-basis on `wire` (forward)."""
    if axis.upper() == "X":
        qml.Hadamard(wires=wire)
    elif axis.upper() == "Y":
        # S† then H maps Y -> Z
        qml.adjoint(qml.S)(wires=wire)
        qml.Hadamard(wires=wire)
    elif axis.upper() == "Z":
        pass
    else:
        raise ValueError("Unknown Pauli axis: " + str(axis))


def _map_axis_to_Z_inverse(axis, wire):
    """Inverse mapping (undo forward)."""
    if axis.upper() == "X":
        qml.Hadamard(wires=wire)
    elif axis.upper() == "Y":
        # inverse of (S†; H) is (H; S)
        qml.Hadamard(wires=wire)
        qml.S(wires=wire)
    elif axis.upper() == "Z":
        pass
    else:
        raise ValueError("Unknown Pauli axis: " + str(axis))


# ---------------- Gate application ----------------
def apply_gate(g, x, α):
    op = g["type"]
    wires = g["targets"]

    # Simple Pauli apply
    if op in ["X", "Y", "Z"]:
        getattr(qml, f"Pauli{op}")(wires=wires[0])
        return

    if op == "H":
        qml.Hadamard(wires=wires[0])
        return

    if op == "CNOT":
        qml.CNOT(wires=wires)
        return

    # Multi-qubit Pauli-rotations parameterized by α[...] or x[...]
    # Expect op like "RXY(α[0])" or "RZZ(x[1])"
    if op.startswith("R") and "(" in op and ")" in op and len(op[1:op.index("(")]) == 2 and all(c in "XYZ" for c in op[1:op.index("(")]):
        pauli_pair = op[1:3]   # e.g. 'XX', 'XY', 'YZ'
        param_tok = op[op.index("(")+1 : op.index(")")]  # 'α[0]' or 'x[1]'

        # parse angle source
        if "α[" in param_tok:
            # encoding gate — decompose to Z basis + CNOT + RZ + CNOT + undo basis
            idx = int(param_tok[param_tok.index("[")+1 : param_tok.index("]")])
            angle = float(α[idx])
            # require exactly two wires
            if len(wires) != 2:
                raise ValueError("Two-qubit Pauli rotations expect exactly 2 target wires.")
            w0, w1 = wires[0], wires[1]
            a0, a1 = pauli_pair[0], pauli_pair[1]

            # map each to Z
            _map_axis_to_Z_forward(a0, w0)
            _map_axis_to_Z_forward(a1, w1)

            # entangle -> RZ(angle) on target (w1) -> disentangle
            qml.CNOT(wires=[w0, w1])
            qml.RZ(angle, wires=w1)
            qml.CNOT(wires=[w0, w1])

            # undo maps
            _map_axis_to_Z_inverse(a1, w1)
            _map_axis_to_Z_inverse(a0, w0)
            return

        elif "x[" in param_tok:
            # fixed gate — use native PauliRot
            idx = int(param_tok[param_tok.index("[")+1 : param_tok.index("]")])
            angle = float(x[idx])
            if len(wires) != 2:
                raise ValueError("Two-qubit Pauli rotations expect exactly 2 target wires.")
            # Pennylane accepts pauli labels like 'XY'
            qml.PauliRot(angle, pauli_pair, wires=wires)
            return

    # Single-qubit encoding gates (α)
    if "α[" in op:
        axis = op[1]
        idx = int(op[op.index("[")+1:op.index("]")])
        angle = α[idx]

        if axis.lower() == "x":
            # Rx(α) = H RZ H
            qml.Hadamard(wires=wires[0])
            qml.RZ(angle, wires=wires[0])
            qml.Hadamard(wires=wires[0])
        elif axis.lower() == "y":
            # Ry(α) = S† H RZ H S
            qml.adjoint(qml.S)(wires=wires[0])
            qml.Hadamard(wires=wires[0])
            qml.RZ(angle, wires=wires[0])
            qml.Hadamard(wires=wires[0])
            qml.S(wires=wires[0])
        else:  # Rz stays as is
            qml.RZ(angle, wires=wires[0])
        return

    # Single-qubit fixed gates (x)
    if "x[" in op:
        axis = op[1]
        idx = int(op[op.index("[")+1:op.index("]")])
        angle = x[idx]
        getattr(qml, f"R{axis.upper()}")(angle, wires=wires[0])
        return

    # fallback — unknown gate
    raise ValueError(f"Unrecognized gate format in apply_gate: {op}")


def build_gate_string_export(g, inverse=False):
    """
    Returns a string representing the gate in code.
    For encoding two-qubit rotations we export the decomposed sequence,
    for fixed two-qubit rotations we export PauliRot.
    """
    name, t = g["type"], g["targets"]
    prefix = "-" if inverse else ""

    # Simple single-qubit gates
    if name in ["X", "Y", "Z"]:
        return f"Pauli{name}(wires={t[0]})"
    if name == "H":
        return f"Hadamard(wires={t[0]})"
    if name == "CNOT":
        return f"CNOT(wires={t})"

    # Multi-qubit pauli rotations like RXY(α[0]) or RYY(x[1])
    if name.startswith("R") and "(" in name and ")" in name and len(name[1:name.index("(")]) == 2 and all(c in "XYZ" for c in name[1:name.index("(")]):
        pauli_pair = name[1:3]
        param_tok = name[name.index("(")+1 : name.index(")")]
        if "α[" in param_tok:
            idx = int(param_tok[param_tok.index("[")+1 : param_tok.index("]")])
            # Export the decomposition as a sequence of qml calls (semicolon-separated)
            w0, w1 = t[0], t[1]
            a0, a1 = pauli_pair[0], pauli_pair[1]
            # forward basis changes
            seq = []
            # forward map a0 -> Z on w0
            if a0 == "X":
                seq.append(f"qml.Hadamard(wires={w0})")
            elif a0 == "Y":
                seq.append(f"qml.adjoint(qml.S)(wires={w0})")
                seq.append(f"qml.Hadamard(wires={w0})")
            # forward map a1 -> Z on w1
            if a1 == "X":
                seq.append(f"qml.Hadamard(wires={w1})")
            elif a1 == "Y":
                seq.append(f"qml.adjoint(qml.S)(wires={w1})")
                seq.append(f"qml.Hadamard(wires={w1})")
            # entangle, RZ, disentangle
            seq.append(f"qml.CNOT(wires=({w0},{w1}))")
            seq.append(f"qml.RZ({prefix}α[{idx}], wires={w1})")
            seq.append(f"qml.CNOT(wires=({w0},{w1}))")
            # undo maps (inverse order)
            if a1 == "X":
                seq.append(f"qml.Hadamard(wires={w1})")
            elif a1 == "Y":
                seq.append(f"qml.Hadamard(wires={w1})")
                seq.append(f"qml.S(wires={w1})")
            if a0 == "X":
                seq.append(f"qml.Hadamard(wires={w0})")
            elif a0 == "Y":
                seq.append(f"qml.Hadamard(wires={w0})")
                seq.append(f"qml.S(wires={w0})")
            return "; ".join(seq)

        elif "x[" in param_tok:
            idx = int(param_tok[param_tok.index("[")+1 : param_tok.index("]")])
            # export native PauliRot call
            return f"PauliRot({prefix}x[{idx}], '{pauli_pair}', wires={t})"

    # Single-qubit α decomp export
    if "α[" in name:
        axis = name[1]
        idx = int(name[name.index("[")+1:name.index("]")])
        param = f"{prefix}α[{idx}]"
        if axis.lower() == "x":
            return f"H(wires={t[0]}); qml.RZ({param}, wires={t[0]}); H(wires={t[0]})  # RX({param})"
        elif axis.lower() == "y":
            return f"qml.RZ(-np.pi/2.0, wires={t[0]}); H(wires={t[0]}); qml.RZ({param}, wires={t[0]}); H(wires={t[0]}); qml.RZ(np.pi/2.0, wires={t[0]})  # RY({param})"
        else:
            return f"RZ({param}, wires={t[0]})"

    # Single-qubit x export unchanged
    if "x[" in name:
        axis = name[1]
        idx = int(name[name.index("[")+1:name.index("]")])
        return f"R{axis.upper()}({prefix}x[{idx}], wires={t[0]})"

    # Fallback
    return name


def show_circuit(b):
    dev = qml.device("default.qubit", wires=n_qubits)
    P = build_P(observable_terms)

    @qml.qnode(dev)
    def U(x=None, α=None):
        if x is None: x = np.zeros(dim_x)
        if α is None: α = np.zeros(dim_alpha)
        for g in gates:
            apply_gate(g, x, α)
        return qml.state()

    @qml.qnode(dev)
    def UPU(x=None, α=None):
        if x is None: x = np.zeros(dim_x)
        if α is None: α = np.zeros(dim_alpha)
        for g in gates:
            apply_gate(g, x, α)
        qml.apply(P)
        for g in reversed(gates):
            apply_gate_inverse(g, x, α)
        return qml.state()

    x0, α0 = np.zeros(dim_x), np.zeros(dim_alpha)

    with output_display:
        clear_output()

        if display_mode.value == "U":
            qml.draw_mpl(U)(x0, α0)
            plt.title("U")
        else:
            qml.draw_mpl(UPU)(x0, α0)
            plt.title("U P U†")
        plt.show()

        print("\n============================")
        print(" Code to recreate this circuit")
        print("============================\n")

        code = ""
        # --- Imports ---
        code += "import pennylane as qml\n"
        code += "import numpy as np\n\n"

        # --- Dimensions and observables ---
        code += f"dim_x = {dim_x}\n"
        code += f"dim_alpha = {dim_alpha}\n"
        code += f"observable_terms = {observable_terms}\n\n"

        # --- build_P function ---
        code += "def build_P(observable_terms):\n"
        code += "    P = None\n"
        code += "    for term in observable_terms:\n"
        code += "        pieces = term.replace('+',' ').split()\n"
        code += "        local = None\n"
        code += "        for p in pieces:\n"
        code += "            op = p[0].upper()\n"
        code += "            idx = int(p[1:])\n"
        code += "            t = getattr(qml, f'Pauli{op}')(idx)\n"
        code += "            local = t if local is None else local @ t\n"
        code += "        P = local if P is None else P + local\n"
        code += "    return P if P is not None else qml.PauliZ(0)\n\n"

        # --- Device ---
        code += f"dev = qml.device('default.qubit', wires={n_qubits})\n\n"

        # --- QNode U ---
        code += "@qml.qnode(dev)\n"
        code += "def U(x, α):\n"
        for g in gates:
            code += f"    qml.{build_gate_string_export(g)}\n"
        code += "    return qml.state()\n\n"

        # --- QNode UPU ---
        if display_mode.value == "U P U†":
            code += "@qml.qnode(dev)\n"
            code += "def UPU(x, α):\n"
            for g in gates:
                code += f"    qml.{build_gate_string_export(g)}\n"
            code += "    P = build_P(observable_terms)\n"
            for g in reversed(gates):
                code += f"    qml.{build_gate_string_export(g, inverse=True)}\n"
            code += "    return qml.state()\n\n"

        # --- Example execution ---
        code += "x0 = np.zeros(dim_x)\n"
        code += "α0 = np.zeros(dim_alpha)\n"
        code += "qml.draw_mpl(U)(x0, α0);\n"
        if display_mode.value == "U P U†":
            code += "qml.draw_mpl(UPU)(x0, α0);\n"

        show_code_block(code)


def apply_gate_inverse(g, x, α):
    op = g["type"]
    wires = g["targets"]

    if op in ["X", "Y", "Z"]:
        getattr(qml, f"Pauli{op}")(wires=wires[0])
        return
    if op == "H":
        qml.Hadamard(wires=wires[0])
        return
    if op == "CNOT":
        qml.CNOT(wires=wires)
        return

    # Two-qubit pauli rotation inverse
    if op.startswith("R") and "(" in op and ")" in op and len(op[1:op.index("(")]) == 2 and all(c in "XYZ" for c in op[1:op.index("(")]):
        pauli_pair = op[1:3]
        param_tok = op[op.index("(")+1 : op.index(")")]
        if "α[" in param_tok:
            idx = int(param_tok[param_tok.index("[")+1 : param_tok.index("]")])
            angle = -float(α[idx])
            if len(wires) != 2:
                raise ValueError("Two-qubit Pauli rotations expect exactly 2 target wires.")
            w0, w1 = wires[0], wires[1]
            a0, a1 = pauli_pair[0], pauli_pair[1]

            # map to Z
            _map_axis_to_Z_forward(a0, w0)
            _map_axis_to_Z_forward(a1, w1)

            qml.CNOT(wires=[w0, w1])
            qml.RZ(angle, wires=w1)
            qml.CNOT(wires=[w0, w1])

            # undo maps
            _map_axis_to_Z_inverse(a1, w1)
            _map_axis_to_Z_inverse(a0, w0)
            return

        elif "x[" in param_tok:
            idx = int(param_tok[param_tok.index("[")+1 : param_tok.index("]")])
            angle = -float(x[idx])
            if len(wires) != 2:
                raise ValueError("Two-qubit Pauli rotations expect exactly 2 target wires.")
            qml.PauliRot(angle, pauli_pair, wires=wires)
            return

    # single-qubit inverse α
    if "α[" in op:
        axis = op[1]
        idx = int(op[op.index("[")+1:op.index("]")])
        angle = - α[idx]

        if axis.lower() == "x":
            qml.Hadamard(wires=wires[0])
            qml.RZ(angle, wires=wires[0])
            qml.Hadamard(wires=wires[0])
        elif axis.lower() == "y":
            qml.adjoint(qml.S)(wires=wires[0])
            qml.Hadamard(wires=wires[0])
            qml.RZ(angle, wires=wires[0])
            qml.Hadamard(wires=wires[0])
            qml.S(wires=wires[0])
        else:
            qml.RZ(angle, wires=wires[0])
        return

    # single-qubit inverse x
    if "x[" in op:
        axis = op[1]
        idx = int(op[op.index("[")+1:op.index("]")])
        angle = - x[idx]
        getattr(qml, f"R{axis.upper()}")(angle, wires=wires[0])
        return

    raise ValueError(f"Unrecognized gate format (inverse): {op}")


# ---------------- Layout ----------------
display(widgets.HTML("<h3>Select dims and qubits first:</h3>"))
display(widgets.HBox([dim_x_widget, dim_alpha_widget, n_qubits_widget, apply_dims_button]), dims_out)

display(gate_category)
display(fixed_gate_widget)
display(encoding_gate_widget)
display(target_qubits_widget)
display(add_gate_button)
display(remove_gate_widget, remove_gate_button)
display(gate_display)

display(widgets.HBox([observable_input, add_obs_button]))
display(widgets.HBox([remove_obs_widget, remove_obs_button]))
display(obs_display)

display(display_mode)
display(show_circuit_button, clear_button)
display(output_display)
exit_button = widgets.Button(description="Exit GUI", button_style="danger")
def exit_gui(b):
    for w in [
        dim_x_widget, dim_alpha_widget, n_qubits_widget, apply_dims_button,
        gate_category, fixed_gate_widget, encoding_gate_widget,
        target_qubits_widget, add_gate_button, remove_gate_widget, remove_gate_button,
        observable_input, add_obs_button, remove_obs_widget, remove_obs_button,
        display_mode, show_circuit_button, clear_button, exit_button,
        gate_display, obs_display, output_display, dims_out
    ]:
        try:
            w.close()
        except:
            pass

    clear_output(wait=True)
    print("GUI closed. You can now run the next cell.")

exit_button.on_click(exit_gui)
display(exit_button)

# Wiring callbacks
apply_dims_button.on_click(apply_dims)
gate_category.observe(update_gate_dropdown, names="value")
add_gate_button.on_click(add_gate_callback)
remove_gate_button.on_click(remove_gate_callback)
add_obs_button.on_click(add_obs_callback)
remove_obs_button.on_click(remove_obs_callback)
show_circuit_button.on_click(show_circuit)
clear_button.on_click(clear_all)


## Part 2

In [ ]:
import ast, math, re, textwrap, ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import numpy as np

# ---------------- State holder ----------------
parsed_holder = {"dim_x":0, "dim_alpha":0, "last_parsed": None}

# ---------------- Widgets ----------------
code_input = widgets.Textarea(
    placeholder="Paste your Python/PennyLane code here...",
    description="Code Input:",
    layout=widgets.Layout(width="100%", height="320px"),
)

count_input = widgets.IntText(
    value=10000,
    description="Shots:",
    layout=widgets.Layout(width="200px")
)

num_X_input = widgets.IntText(
    value=3,
    description="Num X:",
    layout=widgets.Layout(width="200px")
)

submit_button = widgets.Button(
    description="Submit Code",
    button_style="primary"
)

# Linspace UI containers (X) (initially hidden)
x_linspace_box = widgets.VBox()
spec_container = widgets.VBox()
spec_container.layout.display = "none"  # hide until submit

train_test_split_input = widgets.FloatText(
    value=0.2,
    description="Test Ratio:",
    layout=widgets.Layout(width="400px")
)
train_test_split_input.layout.display = "none"
# Alpha as a single comma-separated input (initially hidden)
alpha_text = widgets.Text(
    placeholder="e.g. 0.5, -0.2, 1.3",
    description="α list:",
    layout=widgets.Layout(width="400px")
)
alpha_text.layout.display = "none"
seed_input = widgets.IntText(
    value=42,
    description="Seed:",
    layout=widgets.Layout(width="200px")
)
seed_input.layout.display = "none"
# Generate button (hidden until submit)
generate_button = widgets.Button(
    description="Generate Modified UPU with Registers + AuPAu",
    button_style="success"
)
generate_button.layout.display = "none"

output_display = widgets.Output()

# Display top-level controls: code, shots, num_X, submit
display(code_input, count_input, num_X_input, submit_button, seed_input, train_test_split_input, spec_container, alpha_text, generate_button, output_display)

# ---------------- AST helpers / Gate object ----------------
def ast_to_source(node):
    try:
        return ast.unparse(node)
    except Exception:
        return "<expr>"

class Gate:
    def __init__(self, opname, args_src, wires):
        self.op = opname
        self.args_src = args_src
        self.wires = wires

    def rendered(self, offset=0, replace_angle=None):
        off = [str(w + offset) for w in self.wires]
        op = self.op

        def maybe_replace(s):
            if replace_angle is None:
                return s
            try:
                return replace_angle(s)
            except:
                return s

        # --- NEW: PauliRot support ---
        if op == "PauliRot":
            angle = maybe_replace(self.args_src[0])
            pauli_str = self.args_src[1] if len(self.args_src) > 1 else '"X"'
            wires_code = f"wires=({', '.join(off)})"
            return f"qml.PauliRot({angle}, {pauli_str}, {wires_code})"

        # existing path for RX, RY, RZ
        if op.upper().startswith("R") and len(op) >= 2 and op[1].upper() in ("X","Y","Z"):
            angle = maybe_replace(self.args_src[0] if self.args_src else "?")
            return f"qml.R{op[1].upper()}({angle}, wires={off[0]})"

        # existing ops
        if op.upper() in ("CNOT","CX"):
            return f"qml.CNOT(wires=({off[0]},{off[1]}))"
        if op.upper() in ("H","HADAMARD"):
            return f"qml.Hadamard(wires={off[0]})"

        args = ", ".join([maybe_replace(a) for a in self.args_src]) if self.args_src else ""
        wires_code = f"wires={off[0]}" if len(off)==1 else f"wires=({', '.join(off)})"
        return f"qml.{op}({args}, {wires_code})" if args else f"qml.{op}({wires_code})"


# ---------------- Parsing ----------------
def parse_pennylane_code(code_text):
    tree = ast.parse(code_text)
    imports_src, build_P_src = [], ""
    dim_x, dim_alpha, observable_terms, orig_qubits = None, None, [], None

    for node in tree.body:
        if isinstance(node,(ast.Import,ast.ImportFrom)):
            seg = ast.get_source_segment(code_text,node)
            if seg: imports_src.append(seg)

    def try_eval(node):
        try: return ast.literal_eval(node)
        except: return None

    for node in tree.body:
        if isinstance(node, ast.Assign):
            for t in node.targets:
                if getattr(t,"id","")=="dim_x": dim_x=try_eval(node.value)
                if getattr(t,"id","")=="dim_alpha": dim_alpha=try_eval(node.value)
                if getattr(t,"id","")=="observable_terms": observable_terms=try_eval(node.value)
            if isinstance(node.value, ast.Call):
                func = node.value.func
                if isinstance(func, ast.Attribute) and getattr(func,"attr","")=="device":
                    for kw in node.value.keywords:
                        if kw.arg=="wires":
                            try: orig_qubits=int(ast.literal_eval(kw.value))
                            except: pass

    for node in tree.body:
        if isinstance(node, ast.FunctionDef) and node.name=="build_P":
            seg = ast.get_source_segment(code_text,node)
            if seg: build_P_src=seg

    gates_U, gates_UPU, alpha_usage = [], [], {}

    def process_call(call_node, inside_U=False):
        opname=None
        func=call_node.func
        if isinstance(func,ast.Attribute): opname=func.attr
        elif isinstance(func,ast.Name): opname=func.id
        else: return None

        wires=[]
        for kw in call_node.keywords:
            if kw.arg=="wires":
                v=kw.value
                if isinstance(v, ast.Constant) and isinstance(v.value,int): wires=[v.value]
                elif isinstance(v,(ast.Tuple,ast.List)):
                    wires=[]
                    for elt in v.elts:
                        try: wires.append(int(ast.literal_eval(elt)))
                        except: pass
                else:
                    s=ast_to_source(v)
                    nums=[int(x) for x in re.findall(r"-?\d+",s)]
                    wires=nums
                break
        if not wires and call_node.args:
            last=call_node.args[-1]
            if isinstance(last,(ast.Tuple,ast.List)):
                wires=[]
                for elt in last.elts:
                    try: wires.append(int(ast.literal_eval(elt)))
                    except: pass
        args_src=[]
        for a in call_node.args:
            s=ast_to_source(a)
            args_src.append(s)
            if inside_U:
                m=re.findall(r"(?:α|alpha)\s*\[\s*(\d+)\s*\]", s)
                for mm in m: alpha_usage[int(mm)]=alpha_usage.get(int(mm),0)+1
                m2=re.findall(r"-\s*(?:α|alpha)\s*\[\s*(\d+)\s*\]", s)
                for mm in m2: alpha_usage[int(mm)]=alpha_usage.get(int(mm),0)+1
        if inside_U:
            for kw in call_node.keywords:
                if kw.arg!="wires":
                    s=ast_to_source(kw.value)
                    m=re.findall(r"(?:α|alpha)\s*\[\s*(\d+)\s*\]",s)
                    for mm in m: alpha_usage[int(mm)]=alpha_usage.get(int(mm),0)+1
        return Gate(opname,args_src,wires) if opname else None

    for node in tree.body:
        if isinstance(node, ast.FunctionDef) and node.name in ("U","UPU"):
            inside_U = node.name=="U"
            target = gates_U if inside_U else gates_UPU
            for stmt in node.body:
                if isinstance(stmt,ast.Expr) and isinstance(stmt.value,ast.Call):
                    g=process_call(stmt.value,inside_U=inside_U)
                    if g: target.append(g)

    return imports_src, build_P_src, dim_x, dim_alpha, observable_terms, orig_qubits, gates_U, gates_UPU, alpha_usage

# ---------------- Code generation helpers ----------------
def freq_size_from_count(n):
    return math.ceil(math.log2(2*n+1)) if n>0 else 0
def same_gate_signature(g1,g2):
    if g1.op.upper()!=g2.op.upper(): return False
    if len(g1.wires)!=len(g2.wires): return False
    return all(a==b for a,b in zip(g1.wires,g2.wires))
def gate_depends_on_x(g):
    for s in g.args_src:
        if re.search(r"\bx\s*\[\s*\d+\s*\]",s): return True
    return False
def replace_x_with_x1(s):
    return re.sub(r"\bx\s*\[\s*(\d+)\s*\]", r"x1[\1]", s)

# ---------------- Main code generation ----------------
def generate_output_code(original_code,
                         imports_src, build_P_src,
                         dim_x, dim_alpha, observable_terms, orig_qubits,
                         gates_U, gates_UPU, alpha_usage, shots, X_spec, A_spec, seed, train_test_split):

    dim_x = 0 if dim_x is None else dim_x
    dim_alpha = 0 if dim_alpha is None else dim_alpha
    orig_qubits = 0 if orig_qubits is None else orig_qubits

    freq_sizes={i: 2*freq_size_from_count(alpha_usage.get(i,0)) for i in range(dim_alpha)}
    n_freq=sum(freq_sizes.values())

    n_kernel=1; n_anc=1; n_circ=orig_qubits
    kernel_offset=0; freq_offset=1; anc_offset=1+n_freq; circ_offset=1+n_freq+1
    n_total=n_kernel+n_freq+n_anc+n_circ

    # Precompute start wires for each alpha register
    freq_start=freq_offset
    freq_wires_map={}
    for i in range(dim_alpha):
        size=freq_sizes[i]
        freq_wires_map[i]=list(range(freq_start,freq_start+size))
        freq_start+=size

    # Build X value arrays (linspace only)
    mode_X = X_spec[0]
    if mode_X != "linspace":
        raise ValueError("generate_output_code currently expects X_spec linspace mode")

    _, num_X, x_ranges = X_spec
    # for each dimension build a linspace of length num_X, then form vectors by taking i-th element
    x_grids = [np.linspace(r[0], r[1], num_X) for r in x_ranges] if x_ranges else []
    X_list = []
    for i in range(num_X):
        if dim_x > 0:
            vec = [x_grids[d][i] for d in range(dim_x)]
            X_list.append(f"np.array({vec})")
        else:
            X_list.append("np.array([])")

    # α direct list (A_spec = ("direct", [val, val, ...]))
    if A_spec[0] != "direct":
        raise ValueError("generate_output_code expects A_spec ('direct', list) for alpha")
    a_vals = A_spec[1] or []
    a_vec = [float(v) for v in a_vals]

    # Start building output code
    out=[]
    out.extend(imports_src or ["import pennylane as qml","import numpy as np"])
    out.append("")
    # Inject VGate/VdGate definitions
    out.append(textwrap.dedent("""
    from tqdm import tqdm
    import time
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn.kernel_ridge import KernelRidge
    from sklearn.metrics import mean_squared_error, r2_score
    
    """))
    out.append(f"SEED = {seed}")
    out.append("np.random.seed(SEED)")
    out.append(textwrap.dedent("""
    
    # Wrapper gates for controlled Adders
    class V(qml.operation.Operation):
        num_params = 2
        par_domain = "R"
        def __init__(self, value, wires, mod):
            super().__init__(value, mod, wires=wires)

        def decomposition(self):
            # Actual gate: controlled Adder
            return [qml.Adder(self.parameters[0], x_wires=self.wires, mod=self.parameters[1])]

    # Adjoint version
    class Vdagger(V):
        def __name__(self):
            return "V†"

        def decomposition(self):
            return [qml.Adder(self.parameters[0], x_wires=self.wires, mod=self.parameters[1])]
    Vdagger.__name__ = r"$V^\dagger$"
    """))

    out.append(f"dim_x={dim_x}")
    out.append(f"dim_alpha={dim_alpha}")
    out.append(f"observable_terms={observable_terms!r}")
    out.append("")
    out.append("# build_P function")
    out.append(textwrap.dedent("""
        def build_P(observable_terms):
            P=None
            for term in observable_terms:
                pieces=term.replace('+',' ').split()
                local=None
                for p in pieces:
                    op=p[0].upper()
                    idx=int(p[1:])
                    t=getattr(qml,f'Pauli{op}')(idx)
                    local=t if local is None else local@t
                P=local if P is None else P+local
            return P if P is not None else qml.PauliZ(0)
        
        def build_shifted_P(observable_terms, circ_offset):
            shifted_terms = []
            for term in observable_terms:
                parts = term.replace('+', ' ').split()
                new_parts = []
                for p in parts:
                    op = p[0].upper()
                    idx = int(p[1:])
                    new_parts.append(f"{op}{idx + circ_offset}")
                shifted_terms.append(" ".join(new_parts))
            return build_P(shifted_terms)
            
    """).strip())
    out.append("")
    out.append(f"n_kernel={n_kernel}; freq_register_sizes={freq_sizes}; n_freq={n_freq}")
    out.append(f"n_anc={n_anc}; n_circ={n_circ}; n_total={n_total}")
    out.append(f"kernel_offset={kernel_offset}; freq_offset={freq_offset}")
    out.append(f"anc_offset={anc_offset}; circ_offset={circ_offset}")
    out.append(f"shots = {shots}")
    out.append("")
    out.append("dev=qml.device('default.qubit', wires=n_total, shots = shots, seed = SEED)")
    out.append("dev_analytic = qml.device('default.qubit', wires=n_total, shots=None)  # analytic")
    out.append("")
    out.append("P_shifted = build_shifted_P(observable_terms, circ_offset=circ_offset)")
    out.append("")

    # U
    out.append("def U(x, α):")
    if not gates_U: out.append("    pass")
    else:
        for g in gates_U: out.append("    "+g.rendered(offset=circ_offset))
    out.append("")
    out.append("@qml.qnode(dev_analytic)")
    out.append("def U_return_expval(x, α):")
    out.append("    U(x,α)")
    out.append("    return qml.expval(P_shifted)")
    out.append("")
    # UPU
    out.append("@qml.qnode(dev)")
    out.append("def UPU(x, α):")
    if not gates_U: out.append("    pass")
    else:
        for g in gates_U: out.append("    "+g.rendered(offset=circ_offset))
    out.append("    P = build_shifted_P(observable_terms,circ_offset)")
    adj_gates=[]
    if gates_UPU:
        drop=0
        if len(gates_UPU)>=len(gates_U) and gates_U:
            matches=True
            for i,g in enumerate(gates_U):
                if not same_gate_signature(g,gates_UPU[i]): matches=False; break
            if matches: drop=len(gates_U)
        adj_gates=gates_UPU[drop:]
    if adj_gates:
        out.append("    # user-provided adjoint (trimmed prefix)")
        for g in adj_gates: out.append("    "+g.rendered(offset=circ_offset))
    else:
        out.append("    # auto-generated adjoint")
        for g in reversed(gates_U):
            op=g.op.upper()
            w=[w+circ_offset for w in g.wires]
            if op == "PAULIROT":
                angle = g.args_src[0]
                pauli_str = g.args_src[1]
                wires = [w+circ_offset for w in g.wires]
                neg = f"-({angle})" if not angle.startswith("-") else angle[1:].strip()
                out.append(f"    qml.PauliRot({neg}, {pauli_str}, wires={tuple(wires)})")
            elif op.startswith("R"):
                angle=g.args_src[0].strip()
                neg=f"-({angle})" if not angle.startswith("-") else angle[1:].strip()
                out.append(f"    qml.R{op[1]}({neg}, wires={w[0]})")
            elif op in ("CNOT","CX"): out.append(f"    qml.CNOT(wires=({w[0]},{w[1]}))")
            elif op in ("H","HADAMARD"): out.append(f"    qml.Hadamard(wires={w[0]})")
            else: out.append("    "+g.rendered(offset=circ_offset))
    out.append("    return qml.state()")
    out.append("")

    # AuPAu
    out.append("def AuPAu(x, x1, α):")
    out.append("    qml.Hadamard(wires=0)\n")
    for g in gates_U:
        alpha_matches=re.findall(r"(?:α|alpha)\s*\[\s*(\d+)\s*\]"," ".join(g.args_src))
        w=[w+circ_offset for w in g.wires]
        if alpha_matches:
            for alpha_idx in map(int, alpha_matches):
                freq_wires=freq_wires_map[alpha_idx]  # use precomputed map
                for ww in w: out.append(f"    qml.CNOT(wires=({ww},{anc_offset}))")
                out.append(f"    qml.ctrl(V, control = {anc_offset}, control_values = [0])( value = 1, wires={freq_wires}, mod=2**{freq_sizes[alpha_idx]})")
                out.append(f"    qml.ctrl(Vdagger, control = {anc_offset}, control_values = [1])( value = -1, wires={freq_wires}, mod=2**{freq_sizes[alpha_idx]})")
                for ww in w: out.append(f"    qml.CNOT(wires=({ww},{anc_offset}))")
        elif gate_depends_on_x(g) and g.op.upper().startswith("R"):
            axis=g.op[1]; angle=g.args_src[0]; angle1=replace_x_with_x1(angle)
            out.append(f"    qml.ctrl(qml.R{axis}, control=0, control_values=[1])({angle}, wires={w[0]})")
            out.append(f"    qml.ctrl(qml.R{axis}, control=0, control_values=[0])({angle1}, wires={w[0]})")
        elif g.op == "PauliRot":
            # controlled version like RX/RY/RY case but on the whole Pauli string
            angle = g.args_src[0]
            angle1 = replace_x_with_x1(angle)
            pauli_str = g.args_src[1]
            wires = [w + circ_offset for w in g.wires]

            out.append(f"    qml.ctrl(qml.PauliRot, control=0, control_values=[1])({angle}, {pauli_str}, wires={tuple(wires)})")
            out.append(f"    qml.ctrl(qml.PauliRot, control=0, control_values=[0])({angle1}, {pauli_str}, wires={tuple(wires)})")

        else: out.append("    "+g.rendered(offset=circ_offset))
    # P
    out.append("    P = build_shifted_P(observable_terms,circ_offset)\n")
    # adjoint
    out.append("    # Adjoint – reverse of duplicated U")
    for g in reversed(gates_U):
        alpha_matches=re.findall(r"(?:α|alpha)\s*\[\s*(\d+)\s*\]"," ".join(g.args_src))
        w=[w+circ_offset for w in g.wires]
        if alpha_matches:
            for alpha_idx in map(int, alpha_matches):
                freq_wires=freq_wires_map[alpha_idx]  # precomputed map
                for ww in w: out.append(f"    qml.CNOT(wires=({ww},{anc_offset}))")
                out.append(f"    qml.ctrl(V, control = {anc_offset}, control_values = [1])( value = 1, wires={freq_wires}, mod=2**{freq_sizes[alpha_idx]})")
                out.append(f"    qml.ctrl(Vdagger, control = {anc_offset}, control_values = [0])( value = -1, wires={freq_wires}, mod=2**{freq_sizes[alpha_idx]})")
                for ww in w: out.append(f"    qml.CNOT(wires=({ww},{anc_offset}))")
        elif g.op == "PauliRot":
            angle = g.args_src[0]
            angle1 = replace_x_with_x1(angle)
            pauli_str = g.args_src[1]
            wires = [w + circ_offset for w in g.wires]

            # adjoint swaps control_values AND negates angle
            out.append(f"    qml.ctrl(qml.PauliRot, control=0, control_values=[0])(-({angle1}), {pauli_str}, wires={tuple(wires)})")
            out.append(f"    qml.ctrl(qml.PauliRot, control=0, control_values=[1])(-({angle}), {pauli_str}, wires={tuple(wires)})")

        elif gate_depends_on_x(g) and g.op.upper().startswith("R"):
            axis=g.op[1]; angle=g.args_src[0]; angle1=replace_x_with_x1(angle)
            out.append(f"    qml.ctrl(qml.R{axis}, control=0, control_values=[0])(-{angle1}, wires={w[0]})")
            out.append(f"    qml.ctrl(qml.R{axis}, control=0, control_values=[1])(-{angle}, wires={w[0]})")
        else:
            op=g.op.upper()
            if op.startswith("R"):
                angle=g.args_src[0].strip(); neg=f"-({angle})" if not angle.startswith("-") else angle[1:].strip()
                out.append(f"    qml.R{op[1]}({neg}, wires={w[0]})")
            elif op in ("CNOT","CX"): out.append(f"    qml.CNOT(wires=({w[0]},{w[1]}))")
            elif op in ("H","HADAMARD"): out.append(f"    qml.Hadamard(wires={w[0]})")
            else: out.append("    "+g.rendered(offset=circ_offset))
    out.append("    qml.Hadamard(wires=0)")

    out.append("")
    out.append(textwrap.dedent("""
    @qml.qnode(dev)
    def AuPAu_sample(x, x1, α):
        AuPAu(x, x1, α)   # run your circuit
        return qml.sample(wires=range(n_total))

    
    # Example caller
    x0=np.zeros(dim_x); x1=np.copy(x0); α0=np.zeros(dim_alpha)
    qml.draw_mpl(U_return_expval)(x0, α0); qml.draw_mpl(UPU)(x0, α0); qml.draw_mpl(AuPAu_sample)(x0, x1, α0);
    plt.show();

    
    def conditional_Z_kernel(x, x1, α):
        samples = AuPAu_sample(x, x1, α)
        
        # Extract each register
        kernel = samples[:, kernel_offset]     # wire 0
        anc = samples[:, anc_offset]           # anc wire
        circ = samples[:, circ_offset]         # circ wire

        # Condition: anc=0 and circ=0
        mask = (anc == 0) & (circ == 0)
        conditioned = kernel[mask]

        total = shots
        kept = conditioned.shape[0]

        if kept == 0:
            return {
                "total_shots": total,
                "kept_shots": 0,
                "expectation": None,
            }

        # convert bits to Z eigenvalues (+1 for 0, -1 for 1)
        zvals = 1 - 2*conditioned       # maps 0→+1, 1→−1

        return {
            "total_shots": total,
            "kept_shots": kept,
            "fraction_kept": kept / total,
            "expectation": np.mean(zvals),
        }
        
    # -------------------------------------------------------
    # Generate dataset
    # -------------------------------------------------------
    def generate_labels(X, α):
        Y = np.array([U_return_expval(x_arr, α) for x_arr in X])
        return Y
    
    """))
    # embed the generated X list and α vector into the output code so user can see them
    out.append("# Generated X (list of numpy arrays):")
    out.append(f"X = [{', '.join(X_list)}]")
    out.append(f"alpha = np.array({a_vec})")
    out.append(f"Y = generate_labels(X,alpha)")
    out.append("")
    out.append(f"X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = {train_test_split}, shuffle = True, random_state = SEED)")
    out.append("")
    out.append(textwrap.dedent("""
if dim_x == 1:
    plt.plot(X, Y, ".-")
    plt.xlabel("x")
    plt.ylabel(r"$\langle 0 | U^{\dagger}(x,\\alpha)\, P\, U(x,\\alpha) | 0 \\rangle$")
    plt.show();
    
    
# -------------------------------------------------------
# FUNCTION — compute train kernel (precomputed kernel)
# -------------------------------------------------------
def compute_kernel_matrix_train(X_train, α):
    n = len(X_train)
    K = np.zeros((n, n))

    for i in tqdm(range(n), desc="Computing train kernel"):
        for j in range(n):
            res = conditional_Z_kernel(X_train[i], X_train[j], α)
            K[i, j] = res["expectation"] if res["expectation"] is not None else 0.0

    return K


# -------------------------------------------------------
# FUNCTION — compute test kernel (test × train)
# -------------------------------------------------------
def compute_kernel_matrix_test(X_test, X_train, α):
    n_test = len(X_test)
    n_train = len(X_train)
    K = np.zeros((n_test, n_train))

    for i in tqdm(range(n_test), desc="Computing test kernel"):
        for j in range(n_train):
            res = conditional_Z_kernel(X_test[i], X_train[j], α)
            K[i, j] = res["expectation"] if res["expectation"] is not None else 0.0

    return K

# -------------------------------------------------------
# KERNEL COMPUTATION WITH TIMING
# -------------------------------------------------------
start_train = time.time()
K_train = compute_kernel_matrix_train(X_train, alpha)
end_train = time.time()
print(f"\\nTime to compute TRAIN kernel = {end_train - start_train:.3f} seconds\\n")

start_test = time.time()
K_test = compute_kernel_matrix_test(X_test, X_train, alpha)
end_test = time.time()
print(f"\\nTime to compute TEST kernel = {end_test - start_test:.3f} seconds\\n")



# -------------------------------------------------------
# KERNEL RIDGE REGRESSION
# -------------------------------------------------------
start_krr = time.time()
krr = KernelRidge(alpha=1e-3, kernel='precomputed')
krr.fit(K_train, Y_train)
end_krr = time.time()
print(f"Time for Kernel Ridge Regression fit = {end_krr - start_krr:.3f} seconds\\n")

Y_train_pred = krr.predict(K_train)
Y_test_pred = krr.predict(K_test)

# -------------------------------------------------------
# METRICS
# -------------------------------------------------------
print("Train MSE =", mean_squared_error(Y_train, Y_train_pred))
print("Test MSE  =", mean_squared_error(Y_test, Y_test_pred))

print("Train R2 =", r2_score(Y_train, Y_train_pred))
print("Test R2  =", r2_score(Y_test, Y_test_pred))

# -------------------------------------------------------
# KERNELS
# -------------------------------------------------------
print("")
print("Train Kernel:")
print(K_train)
print("Train-Test Kernel:")
print(K_test)

def show_kernel_heatmap(K, title="Kernel Matrix"):
    plt.figure(figsize=(6, 5))
    plt.imshow(K, aspect='auto')
    plt.colorbar(label='Kernel value')
    plt.title(title)
    plt.xlabel("j index")
    plt.ylabel("i index")
    plt.tight_layout()
    plt.show()

show_kernel_heatmap(K_train, "Train Kernel")
show_kernel_heatmap(K_test, "Test Kernel (test × train)")
# -------------------------------------------------------
# PLOT RESULTS
# -------------------------------------------------------
if dim_x == 1:
    plt.figure(figsize=(10,5))
    plt.scatter([x[0] for x in X_train], Y_train, label="train true", alpha=0.8)
    plt.scatter([x[0] for x in X_train], Y_train_pred, label="train pred", alpha=0.8)

    plt.scatter([x[0] for x in X_test], Y_test, label="test true", alpha=0.8)
    plt.scatter([x[0] for x in X_test], Y_test_pred, label="test pred", alpha=0.8)

    plt.legend()
    plt.xlabel("x")
    plt.ylabel(r"$\langle 0 | U^{\dagger}(x,\\alpha)\, P\, U(x,\\alpha) | 0 \\rangle$")
    plt.title("Kernel Ridge Regression using Quantum Kernel")
    plt.show()

    """
    ))
    
    
    return "\n".join(out)

# ---------------- UI helper: build linspace boxes ----------------
def build_linspace_boxes():
    """Builds x_linspace_box contents from parsed_holder dims."""
    dim_x = parsed_holder.get("dim_x", 0) or 0

    x_boxes = []
    for i in range(dim_x):
        b = widgets.HBox([
            widgets.FloatText(value=0.0, description=f"X{i} min"),
            widgets.FloatText(value=1.0, description=f"X{i} max")
        ])
        x_boxes.append(b)
    x_linspace_box.children = x_boxes

    # update spec container content (alpha input stays separate)
    spec_container.children = [
        widgets.HTML("<b>Linspace Ranges for X</b>"),
        widgets.HTML("<b>X ranges</b>"),
        x_linspace_box,
        widgets.HTML("<div style='margin-top:8px;'>Enter α values as a comma-separated list in the 'α list' box below.</div>")
    ]

# ---------------- Callbacks ----------------
def submit_code(b):
    with output_display:
        clear_output()
        print("Parsing code...")
    code_text = code_input.value
    try:
        parsed = parse_pennylane_code(code_text)
        imports_src, build_P_src, dim_x, dim_alpha, observable_terms, orig_qubits, gates_U, gates_UPU, alpha_usage = parsed
        parsed_holder["dim_x"] = 0 if dim_x is None else dim_x
        parsed_holder["dim_alpha"] = 0 if dim_alpha is None else dim_alpha
        parsed_holder["last_parsed"] = parsed

        # Build linspace boxes now that we know dims
        build_linspace_boxes()

        # Reveal the linspace UI, alpha input, and generate button
        spec_container.layout.display = ""
        train_test_split_input.layout.display = ""
        alpha_text.layout.display = ""
        seed_input.layout.display = ""
        generate_button.layout.display = ""
        with output_display:
            clear_output()
            print(f"Parsed code. Detected dim_x={parsed_holder['dim_x']}, dim_alpha={parsed_holder['dim_alpha']}.")
            print("Please enter the seed, enter the ratio of dataset used for test, set min/max per X-dimension, enter α as comma-separated list, and press Generate.")
    except Exception as e:
        with output_display:
            clear_output()
            print("Error parsing code:", e)

submit_button.on_click(submit_code)

def generate_modified(b):
    code_text = code_input.value
    shots = count_input.value
    num_X = num_X_input.value if (num_X_input.value and num_X_input.value>0) else 1

    try:
        parsed = parsed_holder.get("last_parsed")
        if parsed is None:
            raise RuntimeError("No parsed code available. Click Submit Code first.")

        imports_src, build_P_src, dim_x, dim_alpha, observable_terms, orig_qubits, gates_U, gates_UPU, alpha_usage = parsed

        # collect X ranges from x_linspace_box
        X_ranges = []
        for row in x_linspace_box.children:
            xmin = float(row.children[0].value)
            xmax = float(row.children[1].value)
            X_ranges.append((xmin, xmax))

        # parse α from single comma-separated input
        alpha_raw = alpha_text.value.strip()
        if alpha_raw == "":
            a_vals = []
        else:
            # split by comma and convert to floats, ignore empty tokens
            a_vals = [float(tok) for tok in [t.strip() for t in alpha_raw.split(",")] if tok != ""]

        # sanity check: if dim_alpha > 0, ensure length matches
        if parsed_holder.get("dim_alpha", 0) and len(a_vals) != parsed_holder.get("dim_alpha", 0):
            raise ValueError(f"Expected {parsed_holder.get('dim_alpha')} alpha values (dim_alpha), but got {len(a_vals)}.")

        X_spec = ("linspace", num_X, X_ranges)
        A_spec = ("direct", a_vals)
        
        seed = seed_input.value
        train_test_split = train_test_split_input.value
        
        output_code = generate_output_code(
            code_text,
            imports_src, build_P_src,
            dim_x, dim_alpha, observable_terms, orig_qubits,
            gates_U, gates_UPU, alpha_usage,
            shots,
            X_spec, A_spec, seed, train_test_split
        )

        show_code_block(output_code)

    except Exception as e:
        with output_display:
            clear_output()
            print("Error:", e)

generate_button.on_click(generate_modified)

# ---------------- UI output helper ----------------
def show_code_block(code_text):
    html = f"""
    <textarea id="code_area" style="width:100%; height:400px; background:#f5f5f5; border:1px solid #ccc; font-size:12px; white-space:pre-wrap; overflow:auto;">{code_text}</textarea>
    <br>
    <button onclick="
        var area = document.getElementById('code_area');
        area.select();
        document.execCommand('copy');
    ">Copy to Clipboard</button>
    """
    with output_display:
        clear_output()
        display(HTML(html))

# ---------------- Exit button ----------------
exit_button=widgets.Button(description="Exit GUI", button_style="danger")
def exit_gui(b):
    clear_output(wait=True)
    print("GUI closed. You can now run the next cell.")

exit_button.on_click(exit_gui)
display(exit_button)
